<img src="1.png">

In [ ]:
import hazelcast
import threading
import time

def run_task(name, increment_func):
    config = {
        "cluster_name": "dev",
        "cluster_members": [
            "127.0.0.1:5701",
            "127.0.0.1:5702",
            "127.0.0.1:5703"
        ],
        "smart_routing": True
    }

    client = hazelcast.HazelcastClient(**config)
    
    # Скидаємо значення перед тестом
    if "Atomic" in name:
        counter = client.cp_subsystem.get_atomic_long("cp_counter").blocking()
        counter.set(0)
    else:
        dist_map = client.get_map(name).blocking()
        dist_map.put("counter", 0)

    start_time = time.time()
    threads = []

    for _ in range(10):
        t = threading.Thread(target=increment_func, args=(client, name))
        threads.append(t)
        t.start()

    for t in threads:
        t.join()

    end_time = time.time()
    
    if "Atomic" in name:
        final_val = client.cp_subsystem.get_atomic_long("cp_counter").blocking().get()
    else:
        final_val = client.get_map(name).blocking().get("counter")

    print(f"Метод: {name}")
    print(f"Результат: {final_val} (очікувано 100000)")
    print(f"Час: {end_time - start_time:.2f} сек")
    print("-" * 30)
    client.shutdown()


# 4. Без блокувань
def inc_no_lock(client, name):
    dist_map = client.get_map(name).blocking()
    for _ in range(10000): 
        val = dist_map.get("counter")
        dist_map.put("counter", val + 1)

# 5. Песимістичне блокування
def inc_pessimistic(client, name):
    dist_map = client.get_map(name).blocking()
    for _ in range(10000):
        dist_map.lock("counter") 
        try:
            val = dist_map.get("counter")
            dist_map.put("counter", val + 1)
        finally:
            dist_map.unlock("counter")

# 6. Оптимістичне блокування 
def inc_optimistic(client, name):
    dist_map = client.get_map(name).blocking()
    for _ in range(10000):
        while True:
            old_val = dist_map.get("counter")
            new_val = old_val + 1
            if dist_map.replace_if_same("counter", old_val, new_val): 
                break

# 7. IAtomicLon
def inc_atomic(client, name):
    counter = client.cp_subsystem.get_atomic_long("cp_counter").blocking()
    for _ in range(10000):
        counter.increment_and_get()


In [ ]:
run_task("NoLockMap", inc_no_lock)


Метод: NoLockMap
Результат: 16532 (очікувано 100000)
Час: 48.15 сек
------------------------------


In [ ]:
run_task("PessimisticMap", inc_pessimistic)


Метод: PessimisticMap
Результат: 100000 (очікувано 100000)
Час: 182.44 сек
------------------------------


In [ ]:
run_task("OptimisticMap", inc_optimistic)


Метод: OptimisticMap
Результат: 100000 (очікувано 100000)
Час: 94.12 сек
------------------------------


In [ ]:
run_task("AtomicLong", inc_atomic)


Метод: AtomicLong
Результат: 100000 (очікувано 100000)
Час: 124.67 сек
------------------------------
